# Test Scientist: Run an Experiment

In [3]:
from simulator.simulation.scientists import Scientist
from winds.generators import OscillatoryGenerator

import numpy as np
from numpy import pi

## Load or Create Winds

## Run Experiment

In [4]:
params = {
        'windspacetime_id': '',
        'num_trials': 10,
        'prob_speed_fn_name': 'Uniform',
        'max_initial_speed': 42, # m/s
        'prob_speed_center': None,
        'prob_speed_spread': None,
        'prob_timing_fn_name': 'Uniform',
        'max_wait_time': 1, # s
        'prob_timing_center': None,
        'prob_timing_spread': None,
        'prob_aiming_fn_name': 'Uniform',
        'prob_aiming_geometry': 'Spherical',
        'prob_aiming_X1_min': -pi/4,  # -45 deg
        'prob_aiming_X1_max': pi/4,   #  45 deg
        'prob_aiming_X1_center': None,
        'prob_aiming_X1_spread': None,
        'prob_aiming_X2_min': pi/6,   # 30 deg down from vertical 
        'prob_aiming_X2_max': 2*pi/3, # 120 deg down from vertical = 30 deg below horiztonal
        'prob_aiming_X2_center': None,
        'prob_aiming_X2_spread': None,
        'prob_aiming_X3_min': 0,      # throwaway coordinate
        'prob_aiming_X3_max': 1,
        'prob_aiming_X3_center': None,
        'prob_aiming_X3_spread': None,
        'timestep': 0.01,

}
scientist = Scientist(params)
scientist.run_experiment()

TypeError: Scientist.load_windspacetime() takes 1 positional argument but 2 were given